### Оформить таблицу, которая станет основой рекомендательной системы.

Импортируем библиотеки

In [1]:
import pandas as pd
import psycopg2
import psycopg2.extras
from collections import Counter
import numpy as np

Подключаемся к Metabaase и получаем данные SQL таблиц

In [2]:
def getData():
    # Запрос SQL
    query = '''with courses_per_user as(
                    select
                        c.user_id,
                        count(distinct(ci.resource_id))
                        from final.carts c
                        join final.cart_items ci on c.id = ci.cart_id
                        where c.state = 'successful' and ci.resource_type = 'Course'
                        group by 1 
                        having count(distinct(ci.resource_id)) > 1
                )
                select
                    distinct(cpu.user_id),
                    ci.resource_id
                    from courses_per_user cpu
                    join final.carts c on cpu.user_id = c.user_id
                    join final.cart_items ci on c.id = ci.cart_id
                    where c.state = 'successful' and ci.resource_type = 'Course' '''.format()
    # Данные соединения к Metabase
    conn = psycopg2.connect(
        dbname='skillfactory',
        user='skillfactory',
        host='84.201.134.129',
        password='cCkxxLVrDE8EbvjueeMedPKt',
        port=5432)
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    dict_cur.close()
    conn.close()
    return data
# создаем датафрейм из полученной таблицы
df = pd.DataFrame(getData())

Отсортируем и сгруппирем таблицу по пользователям, а купленные пользователем курсы будут отражаться списком в отдельном столбце

In [3]:
df = df.sort_values(['user_id', 'resource_id'])
grouped_by_user = df.groupby('user_id')['resource_id'].apply(list).reset_index()

Создадим список из всех возможных пар курсов

In [4]:
courses_pairs = []
# цикл по пользователям
for i in range(grouped_by_user['resource_id'].shape[0]):
    # цикл по курсам в списке
    for a in range(len(grouped_by_user['resource_id'].iloc[i])):
        # второй цикл по курсам в списке
        for b in range(a+1, len(grouped_by_user['resource_id'].iloc[i])):
            pair_tuple = grouped_by_user['resource_id'].iloc[i][a], grouped_by_user['resource_id'].iloc[i][b]
            courses_pairs.append(pair_tuple)


С помощью Counter подсчитаем для каждой пары их количество в списке. То есть сколько раз эти курсы покупались вместе и преобразуем Counter в словарь

In [5]:
pairs_counter = Counter(courses_pairs).most_common()
pairs_dict = dict(pairs_counter)

##### Для подсчета количества покупок одного курса с другим создадим рабочую таблицу - матрица, где и в строках и столбцах курсы, а на пересечении столбцов и строк количество покупок вместе.

Для этого нам нужен будет словарь, ключами будут курсы, а значениями: словарь из курсов с количеством покупок вместе с курсом-ключом верхнего словаря

Для итерации создадим список уникальных курсов и уникальных пар курсов

In [6]:
unique_courses = list(df['resource_id'].unique())
unique_courses.sort()
pairs_list = list(pairs_dict.keys())

Создаем рабочую таблицу

In [7]:
courses_dict = dict()
for i in range(len(unique_courses)):
    # для каждого курса создаем словарь с его парами и количеством покупок
    dict_course_times = dict()
    for j in range(len(pairs_list)):
        # если курс из верхнего словаря присутствует в паре добавляем в словарь пару и количество
        if unique_courses[i] == pairs_list[j][0]:
            dict_course_times[pairs_list[j][1]] = pairs_dict[pairs_list[j]]
        elif unique_courses[i] == pairs_list[j][1]:
            dict_course_times[pairs_list[j][0]] = pairs_dict[pairs_list[j]]
        else:
            pass   
    courses_dict[unique_courses[i]] = dict_course_times
courses_df = pd.DataFrame(courses_dict)
courses_df = courses_df.sort_index().fillna(0)

Таблица выглядит вот так:

In [8]:
courses_df.head(10)

,356,357,358,359,360,361,362,363,364,365,...,1182,1184,1185,1186,1187,1188,1198,1199,1200,1201
356,0.0,100.0,0.0,0.0,1.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,1.0,0.0,2.0,0.0,0.0,0.0
357,100.0,0.0,0.0,0.0,4.0,18.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,2.0,0.0,1.0,0.0,0.0,0.0
358,0.0,0.0,0.0,15.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
359,0.0,0.0,15.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
360,1.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0
361,17.0,18.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0
362,0.0,1.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
363,0.0,0.0,0.0,1.0,0.0,0.0,18.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
365,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Создаем копию датафрейма и удаляем лишние столбцы

In [25]:
recommendations_df = courses_df.copy()
recommendations_df['most_recommended_course'] = 0
recommendations_df['bought_together_times'] = 0
recommendations_df['second_most_recommended_course'] = 0
recommendations_df['bought_together_times_second'] = 0
recommendations_df = recommendations_df[['most_recommended_course', 'bought_together_times', 'second_most_recommended_course', 'bought_together_times_second']]

Для каждой строки из рабочей таблицы создаем серию из двух наиболее продаваемых в паре курсов

In [26]:
for i in range(len(unique_courses)):
    two_most_paired = pd.Series(dtype=np.int64)
    two_most_paired = courses_df.iloc[i].sort_values(ascending=False).head(2)
    recommendations_df['most_recommended_course'].iloc[i] = two_most_paired.index[0]
    recommendations_df['second_most_recommended_course'].iloc[i] = two_most_paired.index[1]
    recommendations_df['bought_together_times'].iloc[i] = two_most_paired.iloc[0]
    recommendations_df['bought_together_times_second'].iloc[i] = two_most_paired.iloc[1]

Таблица с рекомендациями выглядит вот так

In [27]:
recommendations_df

,most_recommended_course,bought_together_times,second_most_recommended_course,bought_together_times_second
356,571,103,357,100
357,571,112,356,100
358,570,25,752,19
359,570,16,358,15
360,745,17,516,12
...,...,...,...,...
1188,1141,15,513,11
1198,571,5,912,3
1199,523,2,1115,1
1200,794,1,1104,1


Для определения минимальной границы, при которой будем считать докупленный курс случайностью, ознакомимся с описательными данными количества покупок в паре.

In [28]:
recommendations_df['bought_together_times'].describe()

count    126.000000
mean      79.436508
std      120.688458
min        1.000000
25%       16.250000
50%       37.500000
75%       97.250000
max      797.000000
Name: bought_together_times, dtype: float64

In [36]:
recommendations_df['bought_together_times'].nsmallest(30)

902      1
1182     1
1200     1
1201     1
833      2
866      2
911      2
1199     2
830      3
834      3
836      3
365      4
1160     5
1198     5
364      6
837      6
803      7
835      7
865      7
800      8
862      8
1156     8
863     10
864     10
1129    10
810     13
1124    13
1128    13
1188    15
359     16
Name: bought_together_times, dtype: int64

За минимальную границу возьмем 1-ый квартиль. То есть если курс куплен в паре 16 и менее раз, нужно предложить другой курс. В качестве предложения будем использовать случайно выбранный из наиболее продаваемых курсов. Чтобы определить наиболее продаваемые курсы, также ознакомимся с описательной статистикой по общим продажам курсов.

In [14]:
minimum_paired = 16
courses_sold = df['resource_id'].value_counts()
courses_sold.describe()

count     126.000000
mean      270.428571
std       404.128483
min         2.000000
25%        58.500000
50%       124.000000
75%       350.000000
max      2935.000000
Name: resource_id, dtype: float64

Будем считать наиболее продаваемыми курсами, курсы из 4-го квартиля. То есть, курсы с общим количеством продаж более 350 раз. Таких курсов 32.

In [15]:
top_courses = list(courses_sold[courses_sold.values > 350].index)
len(top_courses)

32

Курсы, которые покупались в паре 16 или менее раз, будем заменять случайно выбранными курсами из наиболее продаваемых курсов.
При этом, если курс, который покупался в паре 16 или менее раз, сам находится в списке наиболее продаваемых, его заменять мы не будем.
Так как, возможно эти курсы могут быть связаны, а продавались они в паре мало, потому что в принципе по курсу, к которому делаем рекомендации, мало продаж.

In [17]:
np.random.seed(1000)
for i in range(recommendations_df.shape[0]): 
    if recommendations_df['bought_together_times'].iloc[i] <= minimum_paired:
        two_courses = np.random.choice(top_courses, size=2, replace=False)
        recommendations_df['most_recommended_course'].iloc[i] = two_courses[0]
        recommendations_df['second_most_recommended_course'].iloc[i] = two_courses[1]
    elif recommendations_df['bought_together_times_second'].iloc[i] <= minimum_paired:
        if recommendations_df['most_recommended_course'].iloc[i] in top_courses:
            selected_courses = top_courses.copy()
            selected_courses.remove(recommendations_df['most_recommended_course'].iloc[i])
            pos = np.random.randint(len(selected_courses))
            recommendations_df['second_most_recommended_course'] = selected_courses[pos]
        else:
            pos = np.random.randint(len(top_courses))
            recommendations_df['second_most_recommended_course'] = top_courses[pos]
    else:
        pass        
        

In [22]:
recommendations_df = recommendations_df[['most_recommended_course', 'second_most_recommended_course']]

In [23]:
recommendations_df

,most_recommended_course,second_most_recommended_course
356,571,357
357,571,356
358,570,752
359,570,358
360,745,516
...,...,...
1188,1141,513
1198,571,912
1199,523,1115
1200,794,1104
